# Homework
> https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md

## 1

In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df_train = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_valid = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [3]:
len(df_train.columns)

19

In [4]:
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## 2

In [5]:
def create_duration(df):
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].map(lambda x: x.total_seconds() / 60)
    return df

In [6]:
df_train = create_duration(df_train)
df_valid = create_duration(df_valid)
df_train['duration'].std(), df_valid['duration'].std()

(42.594351241920904, 42.84210176105097)

## 3

In [33]:
def filter_rows(df):
    return df[(df.duration >= 1) & (df.duration <= 60)]

In [34]:
df_train_filtered = filter_rows(df_train)
df_valid_filtered = filter_rows(df_valid)

In [35]:
len(df_train_filtered) / len(df_train), len(df_valid_filtered) / len(df_valid)

(0.9812202822125979, 0.9800944077722545)

## 4 

In [36]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [37]:
df_train_filtered[categorical] = df_train_filtered[categorical].astype(str)
df_valid_filtered[categorical] = df_valid_filtered[categorical].astype(str)

/tmp/ipykernel_16985/3554844493.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_filtered[categorical] = df_train_filtered[categorical].astype(str)
/tmp/ipykernel_16985/3554844493.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_filtered[categorical] = df_valid_filtered[categorical].astype(str)


In [38]:
dv = DictVectorizer()

train_dicts = df_train_filtered[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_valid_filtered[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [39]:
X_train.get_shape(), X_val.get_shape()

((3009173, 515), (2855951, 515))

## 5

In [43]:
y_train = df_train_filtered.duration.values
y_valid = df_valid_filtered.duration.values

In [44]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
mean_squared_error(y_train, y_pred_train, squared=False)

7.649261027855596

## 6

In [45]:
y_pred_val = lr.predict(X_val)
mean_squared_error(y_valid, y_pred_val, squared=False)

7.8118325109416915